Import libraries

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

Load dataset

In [2]:
df = pd.read_csv('complaints_2015.csv')

Look at dataset

In [3]:
df.head(5)

,Ticket #,Customer Complaint,Date,Time,Received Via,City,State,Zip code,Status,Filing on Behalf of Someone,Description
0,250635,Comcast Cable Internet Speeds,4/22/2015,3:53:50 PM,Internet,Abingdon,Maryland,21009,Closed,No,I have been contacting Comcast Internet Techni...
1,223441,Payment disappear - service got disconnected,4/8/2015,10:22:56 AM,Internet,Acworth,Georgia,30102,Closed,No,Back in January 2015 I made 2 payments: One fo...
2,242732,Speed and Service,4/18/2015,9:55:47 AM,Internet,Acworth,Georgia,30101,Closed,Yes,Our home is located at in Acworth Georgia 3010...
3,277946,Comcast Imposed a New Usage Cap of 300GB that ...,5/7/2015,11:59:35 AM,Internet,Acworth,Georgia,30101,Open,Yes,Comcast in the Atlanta area has just put into ...
4,307175,Comcast not working and no service to boot,5/26/2015,1:25:26 PM,Internet,Acworth,Georgia,30101,Solved,No,I have been a customer of Comcast of some sort...


Look at data type

In [4]:
df.dtypes

Ticket #                       object
Customer Complaint             object
Date                           object
Time                           object
Received Via                   object
City                           object
State                          object
Zip code                        int64
Status                         object
Filing on Behalf of Someone    object
Description                    object
dtype: object

Check if there is missing data

In [5]:
df.isnull().sum()

Ticket #                       0
Customer Complaint             0
Date                           0
Time                           0
Received Via                   0
City                           0
State                          0
Zip code                       0
Status                         0
Filing on Behalf of Someone    0
Description                    0
dtype: int64

Convert column "Date" and create new column "month" and "month_day"

In [6]:
df["Date"] = pd.to_datetime(df["Date"])
df['month'] = df['Date'].dt.month
df['month_day'] = df['Date'].dt.strftime('%m-%d')

Find out at what month is the most complaint

In [7]:
df.groupby('month').count()[['Customer Complaint']].reset_index().\
sort_values(by = 'Customer Complaint',ascending = False).head(5)

,month,Customer Complaint
2,6,1281
0,4,545
1,5,399


Find out on what day is the most complaint

In [8]:
df.groupby('month_day').count()[['Customer Complaint']].reset_index().\
sort_values(by = 'Customer Complaint',ascending = False).head(5).style.background_gradient('Blues')

,month_day,Customer Complaint
84,06-24,219
83,06-23,190
85,06-25,98
86,06-26,55
90,06-30,53


Find out on what day is the least complaint

In [9]:
df.groupby('month_day').count()[['Customer Complaint']].reset_index().\
sort_values(by = 'Customer Complaint').head(5).style.background_gradient('Oranges')

,month_day,Customer Complaint
10,04-11,5
32,05-03,5
4,04-05,6
31,05-02,7
39,05-10,7


Count the total status of complaints

In [10]:
df['Status'].value_counts()

Solved     973
Closed     734
Open       363
Pending    155
Name: Status, dtype: int64

What categories of customer complaint are appeared the most?

In [11]:
df['Customer Complaint'].value_counts()[0:19]

Comcast                     83
Comcast Internet            18
Comcast Data Cap            17
comcast                     13
Comcast Data Caps           11
Data Caps                   11
Comcast Billing             11
Unfair Billing Practices     9
Data Cap                     8
Comcast/Xfinity              8
Comcast data caps            8
Comcast internet             8
Internet speed               8
Comcast data cap             8
Comcast service              6
Comcast Service              6
Billing                      6
Comcast billing              6
COMCAST                      6
Name: Customer Complaint, dtype: int64

Because there are some repititions, we try to re-categorized the customer complaint and found out that Internet is the major complaint

In [12]:
test = pd.DataFrame({'index':range(df.shape[0])})
df['Customer Complaint'] = df['Customer Complaint'].str.lower()
test['Customer_Service']=df['Customer Complaint'].str.extract("(service)")
test['Internet']=df['Customer Complaint'].str.extract("(internet)")
test['Billing']=df['Customer Complaint'].str.extract("(billing)")
test['Payment']=df['Customer Complaint'].str.extract("(charge)")
test['Cable']=df['Customer Complaint'].str.extract("(cable)")
test.shape
test.drop('index',axis=1,inplace=True)
major_complaint = test.notnull().sum().sort_values(ascending=False)
major_complaint

Internet            532
Customer_Service    489
Billing             299
Payment             139
Cable                58
dtype: int64

Which city has the most complaint?

In [13]:
df['State'].value_counts()

Georgia                 289
Florida                 240
California              220
Illinois                164
Tennessee               143
Pennsylvania            130
Michigan                115
Washington               98
Colorado                 80
Maryland                 78
New Jersey               75
Texas                    71
Massachusetts            61
Virginia                 60
Indiana                  59
Oregon                   49
Mississippi              39
Minnesota                33
Alabama                  26
Utah                     22
Arizona                  20
South Carolina           18
District Of Columbia     16
New Mexico               15
Louisiana                13
Delaware                 12
New Hampshire            12
Connecticut              12
West Virginia            11
Kentucky                  7
New York                  6
Arkansas                  6
Maine                     5
Missouri                  4
Ohio                      3
Vermont             

Which city has the most "closed" status?

In [14]:
Closed = df.groupby(["State","Status"]).size().unstack().sort_values(by='Closed', ascending = False).\
fillna(0)

Closed.head(3)

Status,Closed,Open,Pending,Solved
State,,,,
Florida,100.0,35.0,4.0,101.0
Georgia,82.0,35.0,46.0,126.0
California,72.0,47.0,14.0,87.0


Which city has the most "Open" status?

In [15]:
Open = df.groupby(["State","Status"]).size().unstack().sort_values(by='Open', ascending = False).\
fillna(0)

Open.head(3)

Status,Closed,Open,Pending,Solved
State,,,,
California,72.0,47.0,14.0,87.0
Florida,100.0,35.0,4.0,101.0
Georgia,82.0,35.0,46.0,126.0


Which city has the most "Pending" status?

In [16]:
Pending = df.groupby(["State","Status"]).size().unstack().sort_values(by='Pending', ascending = False).\
fillna(0)

Pending.head(3)

Status,Closed,Open,Pending,Solved
State,,,,
Georgia,82.0,35.0,46.0,126.0
California,72.0,47.0,14.0,87.0
Tennessee,39.0,33.0,14.0,57.0


Which city has the most "Solved" status?

In [17]:
Solved = df.groupby(["State","Status"]).size().unstack().sort_values(by='Solved', ascending = False).\
fillna(0)

Solved.head(3)

Status,Closed,Open,Pending,Solved
State,,,,
Georgia,82.0,35.0,46.0,126.0
Florida,100.0,35.0,4.0,101.0
California,72.0,47.0,14.0,87.0


Now I try to implement some NLP cases

all sentences using lower case

In [18]:
df['Description'] = df['Description'].apply (lambda x: " ".join (x.lower () for x in x.split ()))
df['Description']

0       i have been contacting comcast internet techni...
1       back in january 2015 i made 2 payments: one fo...
2       our home is located at in acworth georgia 3010...
3       comcast in the atlanta area has just put into ...
4       i have been a customer of comcast of some sort...
                              ...                        
2220    i am a deaf guy. i have asked att or comcast t...
2221    we purchased our own modem and returned the co...
2222    i had an agreement with comcast agent 1 year f...
2223    a few months ago i was forced to finally call ...
2224    my internet disconnects all of the time and i ...
Name: Description, Length: 2225, dtype: object

Remove all punctuation

In [19]:
df ['Description'] = df['Description'].str.replace (r"""[^\w\s]+""","", regex = True)

df['Description']

0       i have been contacting comcast internet techni...
1       back in january 2015 i made 2 payments one for...
2       our home is located at in acworth georgia 3010...
3       comcast in the atlanta area has just put into ...
4       i have been a customer of comcast of some sort...
                              ...                        
2220    i am a deaf guy i have asked att or comcast to...
2221    we purchased our own modem and returned the co...
2222    i had an agreement with comcast agent 1 year f...
2223    a few months ago i was forced to finally call ...
2224    my internet disconnects all of the time and i ...
Name: Description, Length: 2225, dtype: object

Remove all stop words

In [20]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

# remove stop words
stop = stopwords.words ('english')
df['Description'] = df['Description'].apply (lambda x: " ".join (x for x in x.split () if x not in stop))
df['Description']

0       contacting comcast internet technical support ...
1       back january 2015 made 2 payments one januarys...
2       home located acworth georgia 30101 signed one ...
3       comcast atlanta area put effect unprecendented...
4       customer comcast sort 20 years never issues li...
                              ...                        
2220    deaf guy asked att comcast see provide cable d...
2221    purchased modem returned comcast modem 81213 v...
2222    agreement comcast agent 1 year 3999 50 mg bite...
2223    months ago forced finally call comcast extreme...
2224    internet disconnects time rarely get full 106m...
Name: Description, Length: 2225, dtype: object

Tokenizing

In [21]:
from textblob import TextBlob
#nltk.download('punkt')
TextBlob (df['Description'][3]).words

WordList(['comcast', 'atlanta', 'area', 'put', 'effect', 'unprecendented', 'usage', 'cap', '300gb', 'exceed', 'cap', 'charged', 'extra', '10', 'per', '50gb', 'stream', 'online', 'services', 'standard', 'hd', 'hit', 'cap', 'extremely', 'easily', 'comcast', 'never', 'capped', 'usage', 'year', 'new', 'net', 'neutrality', 'rules', 'went', 'effect', 'feel', 'practices', 'unfair', 'competition', 'atlanta', 'market', 'treat', 'customers', 'however', 'choose', 'right'])

Stemming

In [22]:
from nltk.stem import PorterStemmer
st = PorterStemmer ()
df['Description'][:5].apply (lambda x: " ".join ([st.stem(word) for word in x.split ()]))

0    contact comcast internet technic support last ...
1    back januari 2015 made 2 payment one januari s...
2    home locat acworth georgia 30101 sign one year...
3    comcast atlanta area put effect unprecend usag...
4    custom comcast sort 20 year never issu like pa...
Name: Description, dtype: object

Lemmatizing

In [23]:
from textblob import Word
#nltk.download('wordnet')
#nltk.download('omw-1.4')

df['Description'] = df['Description'].apply (lambda x: " ".join ([Word(word).
                                                     lemmatize () for word in x.split ()]))

df['Description']

0       contacting comcast internet technical support ...
1       back january 2015 made 2 payment one january s...
2       home located acworth georgia 30101 signed one ...
3       comcast atlanta area put effect unprecendented...
4       customer comcast sort 20 year never issue like...
                              ...                        
2220    deaf guy asked att comcast see provide cable d...
2221    purchased modem returned comcast modem 81213 v...
2222    agreement comcast agent 1 year 3999 50 mg bite...
2223    month ago forced finally call comcast extremel...
2224    internet disconnect time rarely get full 106mb...
Name: Description, Length: 2225, dtype: object

Select only 2 important columns to do a prediction later on

In [24]:
df = df[['Description', 'Status']]
df = df[pd.notnull(df['Description'])]

df.head (5)

,Description,Status
0,contacting comcast internet technical support ...,Closed
1,back january 2015 made 2 payment one january s...,Closed
2,home located acworth georgia 30101 signed one ...,Closed
3,comcast atlanta area put effect unprecendented...,Open
4,customer comcast sort 20 year never issue like...,Solved


Create a new column to categorize the status

In [25]:
df['category_id'] = df['Status'].factorize()[0]
df.head(5)

,Description,Status,category_id
0,contacting comcast internet technical support ...,Closed,0
1,back january 2015 made 2 payment one january s...,Closed,0
2,home located acworth georgia 30101 signed one ...,Closed,0
3,comcast atlanta area put effect unprecendented...,Open,1
4,customer comcast sort 20 year never issue like...,Solved,2


Split the data into train and test dataset

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score


train_x, valid_x, train_y, valid_y = train_test_split(df['Description'], df['Status'])

Feature engineering using TF-IDF

In [28]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

tfidf_vect = TfidfVectorizer(analyzer = 'word', token_pattern = r'\w{1,}', max_features = 5000)
tfidf_vect.fit(df['Description'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

Model building and evaluation

In [29]:
model = linear_model.LogisticRegression().fit(xtrain_tfidf, train_y)

LogisticRegression (C = 1.0, class_weight = None, dual = False, fit_intercept = True,
                    intercept_scaling = 1, max_iter = 100, multi_class = "ovr", n_jobs = 1,
                    penalty = 'l2', random_state = None, solver = 'liblinear', tol = 0.0001,
                    verbose = 0, warm_start = False)

accuracy = metrics.accuracy_score (model.predict (xvalid_tfidf), valid_y)
print("Accuracy: ", accuracy)

Accuracy:  0.6894075403949731


Print classification report

In [31]:
print(metrics.classification_report (valid_y, model.predict (xvalid_tfidf),
                                     target_names = df['Status'].unique ()))

              precision    recall  f1-score   support

      Closed       0.59      0.88      0.70       170
        Open       0.38      0.18      0.25        82
      Solved       0.71      0.11      0.19        45
     Pending       0.84      0.83      0.83       260

    accuracy                           0.69       557
   macro avg       0.63      0.50      0.49       557
weighted avg       0.68      0.69      0.65       557



Prediction example

In [33]:
category_id_df = df[['Status', "category_id"]].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Status']].values)

# prediction example
text = ['Comcast solved my problem.']
text_features = tfidf_vect.transform (text)
predictions = model.predict (text_features)
print (text)
print ("  - Predicted as: '{}'".format (id_to_category[predictions[0]]))

['Comcast solved my problem.']
  - Predicted as: 'Closed'
